# Evaluation of MSSW

## MSSW Algorithm

### MSSW Preprocessing

In [1]:
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler


def ptg_for_all(reference_data):
    """
    Calculate all P_tgs from reference data

    :param reference_data: array of shape (#points, #attributes) of reference data
    :return: array of shape (#points, #attribute) of corresponding P_tgs
    """
    column_sum = np.sum(reference_data, axis=0)
    return np.divide(reference_data, column_sum)


def information_utilities_for_all(ptgs):
    """
    Calculate information utility values from P_tgs

    :param ptgs: P_tgs as obtained from ptg_for_all(...)
    :return: array of shape (1, #attributes) of the information utility of each attribute
    """
    entropies = np.divide(scipy.stats.entropy(ptgs, axis=0), ptgs.shape[0])
    information_utilities = np.subtract(1, entropies).reshape((1, entropies.shape[0]))
    return information_utilities


def attribute_weights_for_all(information_utilities):
    """
    Calculate the weights of attributes from information utilities

    :param information_utilities: information utilities as obtained from information_utilities_for_all(...)
    :return: array of shape (1, #attributes) of the attribute weights of each attribute
    """
    attribute_weights = np.divide(information_utilities, np.sum(information_utilities))
    return attribute_weights


def get_attribute_weights_from(reference_data):
    """
    Calculate weights of attributes from reference (benchmark) data

    :param reference_data: array of shape (#points, #attributes)
    :return: array of shape (1, #attributes) of the attribute weights of each attribute
    """
    ptgs = ptg_for_all(reference_data)
    information_utilities = information_utilities_for_all(ptgs)
    attribute_weights = attribute_weights_for_all(information_utilities)
    return attribute_weights


def transform_data_by_attribute_weights(original_data, attribute_weights):
    """
    Transform data by the sqrt of attribute weights

    :param original_data: array of shape (#points, #attributes) to transform
    :param attribute_weights: array of shape (1, #attributes) to use for the transformation
    :return: array of shape (#points, #attributes) of weighted data
    """
    sqrt_attribute_weights = np.sqrt(attribute_weights)
    weighted_data = np.multiply(original_data, sqrt_attribute_weights)
    return weighted_data


def transform_batches_by_attribute_weights(original_batches, attribute_weights):
    """
    Transform multiple batches of data by the sqrt of attribute weights

    :param original_batches: list of arrays of shape (n_i, #attributes), i=batch number, n_i > 1
    :param attribute_weights: array of shape (1, #attributes) of weights to use for the transformation
    :return: list of arrays of shape(n_i, #attributes) of weighted data
    """
    weighted_batches = []
    for original_batch in original_batches:
        weighted_batches.append(transform_data_by_attribute_weights(original_batch, attribute_weights))
    return weighted_batches


def mssw_preprocess(reference_data_batches, testing_data_batches):
    """
    Preprocess data batches through minmax scaling, apply weighting so that Euclidean distance on this weighted data
    becomes the desired entropy-weighted distance on the original data

    :param reference_data_batches: list of arrays of shape (n_r_r, #attributes), r_r=reference batch number,
        n_r_r=#points in this batch
    :param testing_data_batches: list of arrays of shape (n_r_t, #attributes), r_t=testing batch number,
        n_r_t=#points in this batch
    :return: (array of shape (sum(n_r_r) #attributes) of joined reference data, weighted reference batches (same
        structure as reference_data_batches), weighted testing batches (same structure as testing_data_batches))
    """
    joined_reference_data = reference_data_batches[0]
    for reference_batch in reference_data_batches[1:]:
        np.append(joined_reference_data, reference_batch, axis=0)

    scaler = MinMaxScaler()
    scaler.fit(joined_reference_data)
    joined_reference_data = scaler.transform(joined_reference_data)
    reference_data_batches = [scaler.transform(batch) for batch in reference_data_batches]
    testing_data_batches = [scaler.transform(batch) for batch in testing_data_batches]

    attribute_weights = get_attribute_weights_from(joined_reference_data)
    weighted_joined_reference_data = transform_data_by_attribute_weights(joined_reference_data, attribute_weights)
    weighted_reference_batches =\
        [transform_data_by_attribute_weights(batch, attribute_weights) for batch in reference_data_batches]
    weighted_testing_batches =\
        [transform_data_by_attribute_weights(batch, attribute_weights) for batch in testing_data_batches]
    return weighted_joined_reference_data, weighted_reference_batches, weighted_testing_batches


### MSSW

In [2]:
"""
Drift detection algorithm from
[1] Y. Yuan, Z. Wang, and W. Wang,
“Unsupervised concept drift detection based on multi-scale slide windows,”
Ad Hoc Networks, vol. 111, p. 102325, Feb. 2021, doi: 10.1016/j.adhoc.2020.102325.

MSSW is an abbreviation for Multi-Scale Sliding Windows

- Unless specified otherwise, functions in this file work with numpy arrays
- The terms "benchmark data" and "reference data" mean the same thing, default is "reference data"
- The terms "slide data" and "testing data" mean the same thing, default is "testing data"
"""
import inspect
import time
import numpy as np
from sklearn.cluster import KMeans


def obtain_cluster_distances_and_sizes(weighted_sub_window, fitted_kmeans, n_clusters):
    """
    Get the sum of centroid distances and size for clusters formed by fitted_kmeans and weighted_sub_window

    :param weighted_sub_window: array of shape (#points, #attributes) of weighted data
    :param fitted_kmeans: fitted sklearn kmeans object to use for clustering of the weighted_sub_window
    :param n_clusters: number of clusters used to fit the kmeans object
    :return: (array of shape (1, n_clusters) of sums of centroid distances,
    array of shape (1, n_clusters) of cluster sizes)
    """
    centroids = fitted_kmeans.cluster_centers_
    predicted_cluster_labels = fitted_kmeans.predict(weighted_sub_window)

    centroid_distance_sums = np.zeros(n_clusters).reshape((1, n_clusters))
    num_points_in_clusters = np.zeros(n_clusters).reshape((1, n_clusters))
    for cluster_id in range(n_clusters):
        cluster_mask = predicted_cluster_labels == cluster_id
        cluster = weighted_sub_window[cluster_mask]

        num_points_in_clusters[0, cluster_id] = cluster.shape[0]

        centroid = centroids[cluster_id]
        centroid_diffs = np.subtract(cluster, centroid)
        euclideans = np.linalg.norm(centroid_diffs, axis=1)
        sum_euclideans = np.sum(euclideans)
        centroid_distance_sums[0, cluster_id] = sum_euclideans

    return centroid_distance_sums, num_points_in_clusters


def calculate_clustering_statistics(weighted_sub_window, fitted_kmeans, n_clusters):
    """
    Cluster the given weighted_sub_window, and then obtain JSEE, Av_ci for all i, and Av_sr from it

    :param weighted_sub_window: array of shape (#points, #attributes) of weighted data
    :param fitted_kmeans: fitted sklearn kmeans object to use for clustering of the weighted_sub_window
    :param n_clusters: number of clusters used to fit the kmeans object
    :return: (JSEE float, Av_ci array of shape (1, #attributes), Av_sr float)
    """
    centroid_distance_sums, num_points_in_clusters = obtain_cluster_distances_and_sizes(
        weighted_sub_window, fitted_kmeans, n_clusters
    )

    JSEE = np.sum(centroid_distance_sums)
    Av_c = np.divide(centroid_distance_sums, num_points_in_clusters)
    Av_sr = JSEE / weighted_sub_window.shape[0]
    return JSEE, Av_c, Av_sr


def get_s_s(weighted_reference_sub_windows, fitted_kmeans, n_clusters):
    """
    Get S_s = the total average distance sequence of sub-windows in reference (benchmark) data

    :param weighted_reference_sub_windows: list of arrays of shape (n_r, #attributes) of weighted reference data,
        r is the sub-window number, n_r=#points in this sub-window
    :param fitted_kmeans: sklearn kmeans object previously fitted to weighted reference (benchmark) data
    :param n_clusters: number of clusters used to fit the kmeans object
    :return: array of shape (1, len(weighted_reference_sub_windows))
    """
    num_sub_windows = len(weighted_reference_sub_windows)
    s_s = np.zeros(num_sub_windows).reshape((1, num_sub_windows))
    for i, weighted_reference_sub_window in enumerate(weighted_reference_sub_windows):
        _, _, Av_sr = calculate_clustering_statistics(weighted_reference_sub_window, fitted_kmeans, n_clusters)
        s_s[0, i] = Av_sr
    return s_s


def get_moving_ranges(s_s):
    """
    Get moving ranges (MR_i) for each sub-window from S_s

    :param s_s: s_s as obtained from get_s_s(...)
    :return: array of shape (1, len(s_s)-1)
    """
    moving_ranges = np.abs(np.subtract(s_s[:, 1:], s_s[:, :-1]))
    return moving_ranges


def get_mean_s_s_and_mean_moving_ranges(weighted_reference_sub_windows, fitted_kmeans, n_clusters):
    """
    Find the S_s and MR sequences and return their mean

    :param weighted_reference_sub_windows: list of arrays of shape (n_r, #attributes) of weighted reference data,
        r is the sub-window number, n_r=#points in this sub-window
    :param fitted_kmeans: sklearn kmeans object previously fitted to weighted reference (benchmark) data
    :param n_clusters: number of clusters used to fit the kmeans object
    :return: (mean of S_s as float, mean of MR as float)
    """
    s_s = get_s_s(weighted_reference_sub_windows, fitted_kmeans, n_clusters)
    moving_ranges = get_moving_ranges(s_s)
    return np.mean(s_s), np.mean(moving_ranges)


# - function to test for concept drift based on the total average distance from one testing (slide) sub-window
def concept_drift_detected(mean_av_s, mean_mr, weighted_testing_sub_window, fitted_kmeans, n_clusters, coeff):
    """
    Test for concept drift in one weighted testing sub-window

    :param mean_av_s: mean_s_s as obtained from get_mean_s_s_and_mean_moving_ranges(...)
    :param mean_mr: mean_mr as obtained from get_mean_s_s_and_mean_moving_ranges(...)
    :param weighted_testing_sub_window: array of shape (#points, #attributes) of one weighted testing sub-window
    :param fitted_kmeans: sklearn kmeans object previously fitted to weighted reference (benchmark) data
    :param n_clusters: number of clusters used to fit the kmeans object
    :param coeff: drift detection coefficient
    :return: True if drift is detected, False otherwise
    """
    UCL_Av_s = mean_av_s + coeff * mean_mr
    LCL_Av_s = mean_av_s - coeff * mean_mr
    _, _, test_Av_sr = calculate_clustering_statistics(weighted_testing_sub_window, fitted_kmeans, n_clusters)

    return not (LCL_Av_s < test_Av_sr < UCL_Av_s)


def all_drifting_batches(
        reference_data_batches,
        testing_data_batches,
        n_clusters=2,
        n_init=10,
        max_iter=300,
        tol=1e-4,
        random_state=None,
        coeff=2.66
):
    """
    Find all drift locations based on the given reference and testing batches

    :param reference_data_batches: list of arrays of shape (n_r_r, #attributes), r_r=reference batch number,
        n_r_r=#points in this batch
    :param testing_data_batches: list of arrays of shape (n_r_t, #attributes), r_t=testing batch number,
        n_r_t=#points in this batch
    :param n_clusters: desired number of clusters for kmeans
    :param random_state: used to potentially control randomness - see sklearn.cluster.KMeans random_state
    :param coeff: coeff used to detect drift, default=2.66
    :return: a boolean list, length=len(testing_data_batches),
        an entry is True if drift was detected there and False otherwise
    """
    weighted_joined_reference_data, weighted_reference_batches, weighted_testing_batches =\
        mssw_preprocess(reference_data_batches, testing_data_batches)

    fitted_kmeans = KMeans(
        n_clusters=n_clusters,
        n_init=n_init,
        max_iter=max_iter,
        tol=tol,
        random_state=random_state
    ).fit(weighted_joined_reference_data)
    mean_av_s, mean_mr = get_mean_s_s_and_mean_moving_ranges(weighted_reference_batches, fitted_kmeans, n_clusters)

    drifts_detected = []
    for weighted_testing_batch in weighted_testing_batches:
        drifts_detected.append(concept_drift_detected(
            mean_av_s, mean_mr, weighted_testing_batch, fitted_kmeans, n_clusters, coeff))
    return drifts_detected

## MSSW Evaluation Helpers

### MSSW Metrics

In [3]:
def fpr_and_latency_when_averaging(drift_locations, num_test_batches, true_drift_idx):
    """The inputs drift_locations and true_drift_idx are is zero-indexed"""
    fpr = 0
    latency = 1
    drift_locations_arr = np.array(drift_locations)
    signal_locations_before_drift = drift_locations_arr[drift_locations_arr < true_drift_idx]
    signal_locations_not_before_drift = drift_locations_arr[drift_locations_arr >= true_drift_idx]
    num_batches_after_first_drift = num_test_batches - (true_drift_idx + 1)
    drift_detected = False # says whether some drift detection was triggered at or after a drift occurrence

    if len(drift_locations) >= 1:
        if len(signal_locations_before_drift) > 0:
            fpr = len(signal_locations_before_drift) / true_drift_idx
        if len(signal_locations_not_before_drift) > 0:
            first_useful_drift_signal = signal_locations_not_before_drift[0]
            latency = (first_useful_drift_signal - true_drift_idx) / num_batches_after_first_drift
            drift_detected = True

    return fpr, latency, drift_detected

### MSSW Randomness-Robust Evaluation

In [4]:
def all_drifting_batches_randomness_robust(reference_data_batches, testing_data_batches, n_clusters=2, n_init=10,
                                           max_iter=300, tol=1e-4, coeff=2.66, true_drift_idx=2, first_random_state=0,
                                           min_runs=10, std_err_threshold=0.05):
    """
    Repeat running mssw.mssw.all_drifting_batches(...) until the s.e. of metrics from different runs is low enough

    :param n_init:
    :param max_iter:
    :param tol:
    :param reference_data_batches: list of arrays of shape (n_r_r, #attributes), r_r=reference batch number,
        n_r_r=#points in this batch
    :param testing_data_batches: list of arrays of shape (n_r_t, #attributes), r_t=testing batch number,
        n_r_t=#points in this batch
    :param n_clusters: desired number of clusters for kmeans
    :param first_random_state: random states used will be incremented from this one
    :param coeff: coeff used to detect drift, default=2.66
    :param std_err_threshold: threshold to stop executing the mssw algorithm
    :return: a list of lists from all_drifting_batches(...), and the mean and s.e. of FPR and latency
    """
    print('min_runs', min_runs)

    fprs = []
    latencies = []
    runs_results_bool = []
    fpr_std_err = -1
    latency_std_err = -1
    num_runs = 0
    random_state = first_random_state
    while num_runs < min_runs or max(fpr_std_err, latency_std_err) > std_err_threshold:
        drifting_batches_bool = mssw.mssw.all_drifting_batches(
            reference_data_batches,
            testing_data_batches,
            n_clusters=n_clusters,
            n_init=n_init,
            max_iter=max_iter,
            tol=tol,
            random_state=random_state,
            coeff=coeff
        )
        # print('drifting_batches_bool')
        # print(drifting_batches_bool)
        drift_locations = np.arange(len(drifting_batches_bool))[drifting_batches_bool]
        # print('drift_locations')
        # print(drift_locations)
        fpr, latency, _ = fpr_and_latency_when_averaging(
            drift_locations,
            len(testing_data_batches),
            true_drift_idx
        )
        fprs.append(fpr)
        latencies.append(latency)
        runs_results_bool.append(drifting_batches_bool)
        num_runs += 1
        random_state += n_init

        # print('number of runs', num_runs)
        if num_runs >= min_runs:
            fpr_std_err = np.std(fprs) / np.sqrt(len(fprs))
            latency_std_err = np.std(latencies) / np.sqrt(len(latencies))
        # print('fprs', fprs, 's.e.', fpr_std_err)
        # print('latencies', latencies, 's.e.', latency_std_err)

    final_fpr_mean = np.mean(fprs)
    final_latency_mean = np.mean(latencies)
    return runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err

## MSSW Evaluation on Local Datasets

## Synthetic Dataset Locations

In [5]:
abrupt_sea_path = '../Datasets_concept_drift/synthetic_data/abrupt_drift/sea_1_abrupt_drift_0_noise_balanced.arff'
abrupt_agraw1_path = '../Datasets_concept_drift/synthetic_data/abrupt_drift/agraw1_1_abrupt_drift_0_noise_balanced.arff'
abrupt_agraw2_path = '../Datasets_concept_drift/synthetic_data/abrupt_drift/agraw2_1_abrupt_drift_0_noise_balanced.arff'

gradual_sea_paths = [
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_05.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_1.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_5.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_10.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_20.arff'
]

gradual_agraw1_paths = [
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_05.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_1.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_5.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_10.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_20.arff'
]

gradual_agraw2_paths = [
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw2_1_gradual_drift_0_noise_balanced_05.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw2_1_gradual_drift_0_noise_balanced_1.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw2_1_gradual_drift_0_noise_balanced_5.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw2_1_gradual_drift_0_noise_balanced_10.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw2_1_gradual_drift_0_noise_balanced_20.arff'
]

all_sea_data_paths = [abrupt_sea_path] + gradual_sea_paths
all_agraw1_data_paths = [abrupt_agraw1_path] + gradual_agraw1_paths
all_agraw2_data_paths = [abrupt_agraw2_path] + gradual_agraw2_paths

only_numerical_data_paths = [abrupt_sea_path] + gradual_sea_paths
only_mixed_data_paths = [abrupt_agraw1_path] + gradual_agraw1_paths + [abrupt_agraw2_path] + gradual_agraw2_paths

## Obtaining all datasets preprocessed

### Helpers

In [6]:
import numpy as np
import pandas as pd
from scipy.io import arff


def accept_data(file_path):
    """Accept an arff file and return its contents in a pandas dataframe"""
    data = arff.loadarff(file_path)
    df = pd.DataFrame(data[0])
    print('df', df)
    return df


def column_values_to_string(df, columns):
    for column in columns:
        df[column] = df[column].str.decode('utf-8')
    return df


def divide_numeric_categorical(df_x):
    df_x_numeric = df_x.select_dtypes(include=[np.number])
    df_x_categorical = df_x.select_dtypes(exclude=[np.number])
    return df_x_numeric, df_x_categorical


def prepare_df_data(df):
    df_y = column_values_to_string(df[['class']], ['class'])
    df_x = df.drop(columns='class')
    df_x_numeric, df_x_categorical = divide_numeric_categorical(df_x)
    df_x_categorical = column_values_to_string(df_x_categorical, list(df_x_categorical.columns))
    return df_x_numeric.join(df_x_categorical), df_y


def get_clean_df(file_path):
    df = accept_data(file_path)
    df_x, df_y = prepare_df_data(df)

    return df_x, df_y

In [8]:
import sklearn


from category_encoders import TargetEncoder
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split


sea_reference_batches = {}
sea_testing_batches = {}
agraw1_exclude_reference_batches = {}
agraw1_exclude_testing_batches = {}
agraw1_onehot_reference_batches = {}
agraw1_onehot_testing_batches = {}
agraw1_target_reference_batches = {}
agraw1_target_testing_batches = {}
agraw2_exclude_reference_batches = {}
agraw2_exclude_testing_batches = {}
agraw2_onehot_reference_batches = {}
agraw2_onehot_testing_batches = {}
agraw2_target_reference_batches = {}
agraw2_target_testing_batches = {}

for file_path in all_sea_data_paths:
    df_x, df_y = get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    reference_data = df_x_ref.to_numpy()
    testing_data = df_x_test.to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    sea_reference_batches[file_path] = ref_batches
    sea_testing_batches[file_path] = test_batches
    
print('sea')
print(sea_reference_batches)
print(sea_testing_batches)

# agraw1 with categories excluded
for file_path in all_agraw1_data_paths:
    df_x, df_y = get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = divide_numeric_categorical(df_x_test)
    
    reference_data = df_x_ref_num.to_numpy()
    testing_data = df_x_test_num.to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw1_exclude_reference_batches[file_path] = ref_batches
    agraw1_exclude_testing_batches[file_path] = test_batches
    
print('agraw1 exclude')
print(agraw1_exclude_reference_batches)
print(agraw1_exclude_testing_batches)

# agraw1 with categories onehot encoded
for file_path in all_agraw1_data_paths:
    df_x, df_y = get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = divide_numeric_categorical(df_x_test)
    
    ref_index = df_x_ref_cat.index
    test_index = df_x_test_cat.index
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(df_x_ref_cat)
    df_x_ref_cat_transformed = pd.DataFrame(encoder.transform(df_x_ref_cat))
    df_x_test_cat_transformed = pd.DataFrame(encoder.transform(df_x_test_cat))
    df_x_ref_cat_transformed.set_index(ref_index, inplace=True)
    df_x_test_cat_transformed.set_index(test_index, inplace=True)
    
    reference_data = df_x_ref_num.join(df_x_ref_cat_transformed, lsuffix='_num').to_numpy()
    testing_data = df_x_test_num.join(df_x_test_cat_transformed, lsuffix='_num').to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw1_onehot_reference_batches[file_path] = ref_batches
    agraw1_onehot_testing_batches[file_path] = test_batches
    
print('agraw1 onehot')
print(agraw1_onehot_reference_batches)
print(agraw1_onehot_testing_batches)

# agraw1 with categories target encoded
for file_path in all_agraw1_data_paths:
    df_x, df_y = get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = divide_numeric_categorical(df_x_test)
    
    ref_index = df_x_ref_cat.index
    test_index = df_x_test_cat.index
    encoder = TargetEncoder()
    encoder.fit(df_x_ref_cat, df_y_ref)
    df_x_ref_cat_transformed = pd.DataFrame(encoder.transform(df_x_ref_cat))
    df_x_test_cat_transformed = pd.DataFrame(encoder.transform(df_x_test_cat))
    df_x_ref_cat_transformed.set_index(ref_index, inplace=True)
    df_x_test_cat_transformed.set_index(test_index, inplace=True)
    
    reference_data = df_x_ref_num.join(df_x_ref_cat_transformed, lsuffix='_num').to_numpy()
    testing_data = df_x_test_num.join(df_x_test_cat_transformed, lsuffix='_num').to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw1_target_reference_batches[file_path] = ref_batches
    agraw1_target_testing_batches[file_path] = test_batches
    
print('agraw1 target')
print(agraw1_target_reference_batches)
print(agraw1_target_testing_batches)


# agraw2 with categories excluded
for file_path in all_agraw2_data_paths:
    df_x, df_y = get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = divide_numeric_categorical(df_x_test)
    
    reference_data = df_x_ref_num.to_numpy()
    testing_data = df_x_test_num.to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw2_exclude_reference_batches[file_path] = ref_batches
    agraw2_exclude_testing_batches[file_path] = test_batches
    
print('agraw2 exclude')
print(agraw2_exclude_reference_batches)
print(agraw2_exclude_testing_batches)

# agraw1 with categories onehot encoded
for file_path in all_agraw2_data_paths:
    df_x, df_y = get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = divide_numeric_categorical(df_x_test)
    
    ref_index = df_x_ref_cat.index
    test_index = df_x_test_cat.index
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(df_x_ref_cat)
    df_x_ref_cat_transformed = pd.DataFrame(encoder.transform(df_x_ref_cat))
    df_x_test_cat_transformed = pd.DataFrame(encoder.transform(df_x_test_cat))
    df_x_ref_cat_transformed.set_index(ref_index, inplace=True)
    df_x_test_cat_transformed.set_index(test_index, inplace=True)
    
    reference_data = df_x_ref_num.join(df_x_ref_cat_transformed, lsuffix='_num').to_numpy()
    testing_data = df_x_test_num.join(df_x_test_cat_transformed, lsuffix='_num').to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw2_onehot_reference_batches[file_path] = ref_batches
    agraw2_onehot_testing_batches[file_path] = test_batches
    
print('agraw2 onehot')
print(agraw2_onehot_reference_batches)
print(agraw2_onehot_testing_batches)

# agraw2 with categories target encoded
for file_path in all_agraw2_data_paths:
    df_x, df_y = get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = divide_numeric_categorical(df_x_test)
    
    ref_index = df_x_ref_cat.index
    test_index = df_x_test_cat.index
    encoder = TargetEncoder()
    encoder.fit(df_x_ref_cat, df_y_ref)
    df_x_ref_cat_transformed = pd.DataFrame(encoder.transform(df_x_ref_cat))
    df_x_test_cat_transformed = pd.DataFrame(encoder.transform(df_x_test_cat))
    df_x_ref_cat_transformed.set_index(ref_index, inplace=True)
    df_x_test_cat_transformed.set_index(test_index, inplace=True)
    
    reference_data = df_x_ref_num.join(df_x_ref_cat_transformed, lsuffix='_num').to_numpy()
    testing_data = df_x_test_num.join(df_x_test_cat_transformed, lsuffix='_num').to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw2_target_reference_batches[file_path] = ref_batches
    agraw2_target_testing_batches[file_path] = test_batches
    
print('agraw2 target')
print(agraw2_target_reference_batches)
print(agraw2_target_testing_batches)

df         attrib1   attrib2   attrib3      class
0      7.308782  4.100808  2.077148  b'groupB'
1      5.833539  0.422983  7.616747  b'groupA'
2      1.397627  6.949480  8.052278  b'groupB'
3      2.750299  0.753878  6.105915  b'groupA'
4      2.049135  6.233638  1.847071  b'groupB'
...         ...       ...       ...        ...
99995  4.636430  6.639370  0.066740  b'groupB'
99996  4.251993  3.351235  5.652197  b'groupA'
99997  4.131405  6.371722  3.125554  b'groupB'
99998  1.404214  4.392506  9.298558  b'groupA'
99999  7.231749  8.770465  3.925490  b'groupB'

[100000 rows x 4 columns]


C:\Users\jpohl\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\jpohl\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df         attrib1   attrib2   attrib3      class
0      7.308782  4.100808  2.077148  b'groupB'
1      5.833539  0.422983  7.616747  b'groupA'
2      1.397627  6.949480  8.052278  b'groupB'
3      2.750299  0.753878  6.105915  b'groupA'
4      2.049135  6.233638  1.847071  b'groupB'
...         ...       ...       ...        ...
99995  2.074508  1.775662  1.318589  b'groupA'
99996  4.636430  6.639370  0.066740  b'groupB'
99997  4.251993  3.351235  5.652197  b'groupA'
99998  4.131405  6.371722  3.125554  b'groupB'
99999  1.404214  4.392506  9.298558  b'groupA'

[100000 rows x 4 columns]


C:\Users\jpohl\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\jpohl\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df         attrib1   attrib2   attrib3      class
0      7.308782  4.100808  2.077148  b'groupB'
1      5.833539  0.422983  7.616747  b'groupA'
2      1.397627  6.949480  8.052278  b'groupB'
3      2.750299  0.753878  6.105915  b'groupA'
4      2.049135  6.233638  1.847071  b'groupB'
...         ...       ...       ...        ...
99995  2.261206  1.404770  3.977088  b'groupA'
99996  0.795885  8.915077  6.892585  b'groupB'
99997  0.353597  1.289198  0.001943  b'groupA'
99998  6.540503  5.698432  7.743243  b'groupB'
99999  2.074508  1.775662  1.318589  b'groupA'

[100000 rows x 4 columns]


...

df               salary    commission   age     elevel       car      zipcode  \
0       58227.414161  42921.435776  77.0  b'level2'   b'car6'  b'zipcode5'   
1       55628.982351  22363.143593  27.0  b'level3'  b'car17'  b'zipcode3'   
2      146598.421289      0.000000  26.0  b'level2'  b'car13'  b'zipcode5'   
3       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
4       83888.656723      0.000000  72.0  b'level3'  b'car11'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   68222.567612  33141.251698  41.0  b'level2'   b'car8'  b'zipcode1'   
99996   50428.438359  59324.405616  26.0  b'level4'   b'car1'  b'zipcode2'   
99997  112914.978798      0.000000  41.0  b'level1'   b'car4'  b'zipcode4'   
99998   90178.866228      0.000000  60.0  b'level1'  b'car13'  b'zipcode6'   
99999   37082.251910  22468.199196  43.0  b'level2'  b'car20'  b'zipcode1'   

             hvalue  hyears           loan      class  
0   

C:\Users\jpohl\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\jpohl\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jpohl\anaconda3\lib\site-packages\category_encoders\target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
C:\Users\jpohl\anaconda3\lib\site-packages\category_encoders\target_encoder.py:129: Fu

df               salary    commission   age     elevel       car      zipcode  \
0       58227.414161  42921.435776  77.0  b'level2'   b'car6'  b'zipcode5'   
1       55628.982351  22363.143593  27.0  b'level3'  b'car17'  b'zipcode3'   
2      146598.421289      0.000000  26.0  b'level2'  b'car13'  b'zipcode5'   
3       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
4       83888.656723      0.000000  72.0  b'level3'  b'car11'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   80507.066152      0.000000  59.0  b'level1'   b'car8'  b'zipcode7'   
99996   43953.844702  23112.195013  36.0  b'level1'  b'car19'  b'zipcode6'   
99997  127958.679326      0.000000  59.0  b'level2'   b'car8'  b'zipcode7'   
99998   58150.729498  54772.811468  66.0  b'level3'  b'car15'  b'zipcode8'   
99999  120616.756315      0.000000  59.0  b'level2'   b'car3'  b'zipcode3'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\jpohl\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jpohl\anaconda3\lib\site-packages\category_encoders\target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
C:\Users\jpohl\anaconda3\lib\site-packages\category_encoders\target_encoder.py:129: Fu

agraw2 target
{'../Datasets_concept_drift/synthetic_data/abrupt_drift/agraw2_1_abrupt_drift_0_noise_balanced.arff': [array([[5.82274142e+04, 4.29214358e+04, 7.70000000e+01, ...,
        5.00332226e-01, 4.81505516e-01, 5.08792497e-01],
       [5.56289824e+04, 2.23631436e+04, 2.70000000e+01, ...,
        4.96433903e-01, 5.04279131e-01, 5.12541553e-01],
       [1.46598421e+05, 0.00000000e+00, 2.60000000e+01, ...,
        5.00332226e-01, 4.92203390e-01, 5.08792497e-01],
       ...,
       [2.48369729e+04, 7.16794719e+04, 3.90000000e+01, ...,
        5.03981027e-01, 5.04279131e-01, 5.12541553e-01],
       [6.03088864e+04, 6.70882404e+04, 5.20000000e+01, ...,
        5.03981027e-01, 4.95765472e-01, 5.06815366e-01],
       [6.50239030e+04, 4.07706319e+04, 5.90000000e+01, ...,
        4.96433903e-01, 4.95675316e-01, 4.97927768e-01]]), array([[1.28572172e+05, 0.00000000e+00, 6.30000000e+01, ...,
        4.99582847e-01, 5.07061197e-01, 4.81747457e-01],
       [3.28746914e+04, 4.94224358e+04, 4.9

## Helpers to obtain valuable information from a kmeans verbose run

In [9]:
import csv
import sys


def write_verbose_kmeans_to_file(result_filename, data_to_cluster, n_clusters, n_init, max_iter, tol, random_state):
    print('random state:', random_state)
    orig_stdout = sys.stdout
    sys.stdout = open(result_filename, 'wt')

    fitted_kmeans = KMeans(
            n_clusters=2,
            n_init=n_init,
            max_iter=max_iter,
            tol=tol,
            verbose=3,
            random_state=random_state
        ).fit(data_to_cluster)

    sys.stdout = orig_stdout
print('something')


def convert_kmeans_output_file_to_dicts(file_path, n_init):
    # read the verbose output file to be able to reach conclusions
    with open(file_path, newline='') as f:
        rdr = csv.reader(f)
        kmeans_verbose_output_list = list(rdr)

    run_results_list = []
    reversed_run_results_list = []

    current_run_reversed_list_messages = []
    for el in reversed(kmeans_verbose_output_list):
        current_run_reversed_list_messages.append(el)
        if el[0] == 'Initialization complete':
            reversed_run_results_list.append(current_run_reversed_list_messages)
            current_run_reversed_list_messages = []

    for reversed_run_result in reversed_run_results_list:
        run_result = reversed_run_result.copy()
        run_result.reverse()
        run_results_list.append(run_result)

    run_results_list.reverse()

    run_results_dicts = []
    for i in range(n_init):
        result_dict = {'converged': False, 'convergence_dict': {}, 'iterations_inertia': []}
        current_result = run_results_list[i]
        only_iterations = current_result[1:]
        if len(current_result[-1]) == 1: # this run converged
            result_dict['converged'] = True
            only_iterations = only_iterations[:-1]
            converge_message_split = current_result[-1][0].split(' ')
            result_dict['convergence_dict']['iteration'] = int(converge_message_split[3][:-1])
            if len(converge_message_split) == 6:
                result_dict['convergence_dict']['type'] = 'strict'
            else:
                result_dict['convergence_dict']['type'] = 'tol-based'
                result_dict['convergence_dict']['center_shift'] = converge_message_split[6]
                result_dict['convergence_dict']['within_tol'] = converge_message_split[9]

        iterations_inertia = []
        for iteration_message_list in only_iterations:
            inertia = float(iteration_message_list[1].split(' ')[2])
            iterations_inertia.append(inertia)
        result_dict['iterations_inertia'] = iterations_inertia

        run_results_dicts.append(result_dict)

    return run_results_dicts
    

def print_stats_from_kmeans_output_dicts(run_results_dicts):
    max_iterations = -1
    initial_inertia = []
    final_inertia = []
    num_convergences = 0
    num_strict_convergences = 0
    num_tol_based_convergences = 0
    for result_dict in run_results_dicts:
        num_iterations = len(result_dict['iterations_inertia'])
        max_iterations = num_iterations if num_iterations > max_iterations else max_iterations
        initial_inertia.append(result_dict['iterations_inertia'][0])
        final_inertia.append(result_dict['iterations_inertia'][-1])
        if result_dict['converged'] == True:
            num_convergences += 1
            if result_dict['convergence_dict']['type'] == 'strict':
                num_strict_convergences += 1
            else:
                num_tol_based_convergences += 1

    print('total number of results:', len(run_results_dicts))
    print('maximum number of iterations:', max_iterations)
    print('minimum initial inertia:', min(initial_inertia))
    print('maximum initial inertia:', max(initial_inertia))
    print('number of unique final inertia values:', len(np.unique(final_inertia)))
    print('minimum final inertia:', min(final_inertia))
    print('maximum final inertia:', max(final_inertia))
    print('total number of convergences:', num_convergences)
    print('number of strict convergences:', num_strict_convergences)
    print('number of tol-based convergences:', num_tol_based_convergences)

something


## Finding the best tol and max_iter in SEA

In [10]:
filename = 'sea_output.txt'
n_init = 100
random_state=1053
ref_batches = sea_reference_batches[abrupt_sea_path]
test_batches = sea_testing_batches[abrupt_sea_path]
weighted_joined_reference_data, _, _ = mssw_preprocess(ref_batches, test_batches)
write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=n_init, max_iter=500, tol=0, random_state=random_state)
output_dicts = convert_kmeans_output_file_to_dicts(filename, n_init=n_init)
print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 54
minimum initial inertia: 657.5117945560726
maximum initial inertia: 1654.92779186788
number of unique final inertia values: 18
minimum final inertia: 597.8461904607494
maximum final inertia: 605.4169667607526
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


### Using the Obtained Parameters to Evaluate the Algorithm on SEA Abrupt

In [ ]:
sys.stdout = orig_stdout
sea_abrupt_all_drifting_batches = all_drifting_batches(sea_abrupt_ref_batches,
        sea_abrupt_test_batches,
        n_clusters=2,
        n_init=100,
        max_iter=500,
        tol=1e-7,
        random_state=0,
        coeff=2.66)

print('sea_abrupt_all_drifting_batches', sea_abrupt_all_drifting_batches)

### Using the Obtained Parameters to Evaluate the Algorithm on All SEA Datasets

In [ ]:
sea_all_drifting_batches = {}
for sea_path in all_sea_data_paths:
    sea_ref_batches = sea_reference_batches[sea_path]
    sea_test_batches = sea_testing_batches[sea_path]
    
    sea_all_drifting_batches[sea_path] = all_drifting_batches(sea_ref_batches,
        sea_test_batches,
        n_clusters=2,
        n_init=100,
        max_iter=500,
        tol=1e-7,
        random_state=0,
        coeff=2.66)
    
print(sea_all_drifting_batches)

## Finding the best tol and max_iter in AGRAW1 Exclude

In [11]:
filename = 'agraw1_exclude_output.txt'
n_init = 100
random_state=1053
ref_batches = agraw1_exclude_reference_batches[abrupt_agraw1_path]
test_batches = agraw1_exclude_testing_batches[abrupt_agraw1_path]
weighted_joined_reference_data, _, _ = mssw_preprocess(ref_batches, test_batches)
write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=n_init, max_iter=500, tol=0, random_state=random_state)
output_dicts = convert_kmeans_output_file_to_dicts(filename, n_init=n_init)
print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 24
minimum initial inertia: 769.9567074502496
maximum initial inertia: 1491.5073913725237
number of unique final inertia values: 1
minimum final inertia: 568.2804090734008
maximum final inertia: 568.2804090734008
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


## Finding the best tol and max_iter in AGRAW1 Onehot

In [12]:
filename = 'agraw1_onehot_output.txt'
n_init = 100
random_state=1053
ref_batches = agraw1_onehot_reference_batches[abrupt_agraw1_path]
test_batches = agraw1_onehot_testing_batches[abrupt_agraw1_path]
weighted_joined_reference_data, _, _ = mssw_preprocess(ref_batches, test_batches)
write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=n_init, max_iter=500, tol=0, random_state=random_state)
output_dicts = convert_kmeans_output_file_to_dicts(filename, n_init=n_init)
print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 19
minimum initial inertia: 1323.2543959360253
maximum initial inertia: 1451.5853803812658
number of unique final inertia values: 18
minimum final inertia: 728.4372189121426
maximum final inertia: 747.0839411450996
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


## Finding the best tol and max_iter in AGRAW1 Target

In [13]:
filename = 'agraw1_target_output.txt'
n_init = 100
random_state=1053
ref_batches = agraw1_target_reference_batches[abrupt_agraw1_path]
test_batches = agraw1_target_testing_batches[abrupt_agraw1_path]
weighted_joined_reference_data, _, _ = mssw_preprocess(ref_batches, test_batches)
write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=n_init, max_iter=500, tol=0, random_state=random_state)
output_dicts = convert_kmeans_output_file_to_dicts(filename, n_init=n_init)
print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 30
minimum initial inertia: 938.6378356982315
maximum initial inertia: 1677.6132171746485
number of unique final inertia values: 13
minimum final inertia: 738.6524992884706
maximum final inertia: 792.4147804590955
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


## Finding the best tol and max_iter in AGRAW2 Exclude

In [14]:
filename = 'agraw2_exclude_output.txt'
n_init = 100
random_state=1053
ref_batches = agraw2_exclude_reference_batches[abrupt_agraw2_path]
test_batches = agraw2_exclude_testing_batches[abrupt_agraw2_path]
weighted_joined_reference_data, _, _ = mssw_preprocess(ref_batches, test_batches)
write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=n_init, max_iter=500, tol=0, random_state=random_state)
output_dicts = convert_kmeans_output_file_to_dicts(filename, n_init=n_init)
print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 27
minimum initial inertia: 773.212821848988
maximum initial inertia: 1541.4774421685145
number of unique final inertia values: 6
minimum final inertia: 593.2816577731598
maximum final inertia: 593.281665957085
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


## Finding the best tol and max_iter in AGRAW2 Onehot

In [15]:
filename = 'agraw2_onehot_output.txt'
n_init = 100
random_state=1053
ref_batches = agraw2_onehot_reference_batches[abrupt_agraw2_path]
test_batches = agraw2_onehot_testing_batches[abrupt_agraw2_path]
weighted_joined_reference_data, _, _ = mssw_preprocess(ref_batches, test_batches)
write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=n_init, max_iter=500, tol=0, random_state=random_state)
output_dicts = convert_kmeans_output_file_to_dicts(filename, n_init=n_init)
print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 29
minimum initial inertia: 1314.4539431527146
maximum initial inertia: 1437.617718127152
number of unique final inertia values: 19
minimum final inertia: 730.5627872414303
maximum final inertia: 750.120304150691
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


## Finding the best tol and max_iter in AGRAW2 Target

In [16]:
filename = 'agraw2_target_output.txt'
n_init = 100
random_state=1053
ref_batches = agraw2_target_reference_batches[abrupt_agraw2_path]
test_batches = agraw2_target_testing_batches[abrupt_agraw2_path]
weighted_joined_reference_data, _, _ = mssw_preprocess(ref_batches, test_batches)
write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=n_init, max_iter=500, tol=0, random_state=random_state)
output_dicts = convert_kmeans_output_file_to_dicts(filename, n_init=n_init)
print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 21
minimum initial inertia: 888.8466859654951
maximum initial inertia: 1776.891247174513
number of unique final inertia values: 5
minimum final inertia: 685.5043938637058
maximum final inertia: 760.4975209431744
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0
